<a href="https://colab.research.google.com/github/informatykazyciem/d3/blob/main/Zal_01_Rapacz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. 

Celem analizy jest dowiedzenie się czy istnieją jakieś związki pomiędzy 3 książkami Juliusza Verne'a.: "Podróż na tajemniczą wyspę", "Podróż do wnętrza ziemi", "W 80 dni dookoła świata". Przeanalizowane zostaną najczęśniej występujące słowa w celu sprawdzenie jak wiele można się z nich dowiedzieć o fabule bez czytania książek, zostanie również sprawdzona tematyka rozdziałów, ich klasyfikacja oraz jaki nastrój w nich panuje. 
Książki zostały pobrane ze strony internetowej https://www.gutenberg.org/. 

In [161]:
import re
import requests
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt
import urllib.request
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from gensim.parsing.preprocessing import remove_stopwords
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from wordcloud import WordCloud
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
import gensim
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer






[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Pobieranie plików tekstów książek do analizy

In [162]:

####The Mysterious Island, by Jules Verne ############
url = "https://www.gutenberg.org/files/1268/1268-0.txt"
plik_wyjsciowy = "plik.txt"

try:
    response = urllib.request.urlopen(url)
    data_MI = response.read().decode("utf-8")
    
    with open(plik_wyjsciowy, "w") as file:
        file.write(data_MI)
    
    print("Plik został pomyślnie pobrany.")
except Exception as e:
    print("Wystąpił błąd podczas pobierania pliku:", str(e))

########### A Journey to the Centre of the Earth, by Jules Verne #############
url = "https://www.gutenberg.org/files/18857/18857.txt"
plik_wyjsciowy = "plik.txt"

try:
    response = urllib.request.urlopen(url)
    data_CE = response.read().decode("utf-8")
    
    with open(plik_wyjsciowy, "w") as file:
        file.write(data_CE)
    
    print("Plik został pomyślnie pobrany.")
except Exception as e:
    print("Wystąpił błąd podczas pobierania pliku:", str(e))

  ################Around the World in Eighty Days, by Jules Verne###############

url = "https://www.gutenberg.org/files/103/103-0.txt"
plik_wyjsciowy = "plik.txt"

try:
    response = urllib.request.urlopen(url)
    data_AW = response.read().decode("utf-8")
    
    with open(plik_wyjsciowy, "w") as file:
        file.write(data_AW)
    
    print("Plik został pomyślnie pobrany.")
except Exception as e:
    print("Wystąpił błąd podczas pobierania pliku:", str(e))

Plik został pomyślnie pobrany.
Plik został pomyślnie pobrany.
Plik został pomyślnie pobrany.


# II. Przygotowanie korpusu

W tej części zostaną przygotowane teksty książek do tego, aby następnie utworzyć z nich chmury słów. Najpierw tekst książki podzielony jest na zdania, a następnie przeprowadzone zostaną: normalizacja, tokenizacja, stemming, lematyzacja. Na końcu tworzony jest korpus- macierz wystąpień.

a) The Mysterious Island


 1.

In [163]:
def rozdziel_na_zdania(tekst):
    # Wyrażenie regularne do wyszukiwania zdań
    wzorzec = r"([^.!?]+[.!?])"
    # Użycie wyrażenia regularnego do podziału tekstu na zdania
    zdania = re.findall(wzorzec, tekst)
    
    return zdania

# Tekst ksiażki
tekst = data_MI

# Wywołanie funkcji rozdzielającej na zdania
zdania = rozdziel_na_zdania(tekst)



Normalizacja tekstu

In [164]:
def czysc_tekst(zdania):
    
    temp = re.sub("\s{2,}", " ", zdania)
    temp = re.sub("(\r\n|\r|\n)", " ", temp) 
    temp = temp.lower() 
    temp = re.sub("rt", "", temp) 
    temp = re.sub("&amp", "", temp) 
    temp = re.sub("#[a-z,A-Z]*", "", temp)
    temp = re.sub("@\w+", "", temp) 
    temp = re.sub("(f|ht)(tp)([^ ]*)", "", temp) 
    temp = re.sub("http(s?)([^ ]*)", "", temp)
    temp = re.sub("[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", " ", temp) 
    temp = re.sub("\d", "", temp) 
    temp = re.sub("\s{2,}", " ", temp) 
    temp = temp.strip()
    return temp
    
czysty = [czysc_tekst(el) for el in zdania]

Tokenizacja


In [165]:

tokenized_sentences = [simple_preprocess(sentence) for sentence in czysty]

# Wyświetlenie wyników tokenizacji
print(tokenized_sentences)

[['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'mysterious', 'island', 'by', 'jules', 'verne', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'pas', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever'], ['you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www'], ['gutenberg'], ['org'], ['if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook'], ['title', 'the', 'mysterious', 'island', 'author', 'jules', 'verne', 'release', 'date', 'april', 'ebook', 'most', 'recently', 'updated', 'june', 'language', 'english', 'character', 'set', 'encoding

Stemming

In [166]:

stemmer = PorterStemmer()
stemmed_sentences = [[stemmer.stem(word) for word in sentence] for sentence in tokenized_sentences]

print(stemmed_sentences)

[['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'mysteri', 'island', 'by', 'jule', 'vern', 'thi', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyon', 'anywher', 'in', 'the', 'unit', 'state', 'and', 'most', 'other', 'pa', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrict', 'whatsoev'], ['you', 'may', 'copi', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'term', 'of', 'the', 'project', 'gutenberg', 'licens', 'includ', 'with', 'thi', 'ebook', 'or', 'onlin', 'at', 'www'], ['gutenberg'], ['org'], ['if', 'you', 'are', 'not', 'locat', 'in', 'the', 'unit', 'state', 'you', 'will', 'have', 'to', 'check', 'the', 'law', 'of', 'the', 'countri', 'where', 'you', 'are', 'locat', 'befor', 'use', 'thi', 'ebook'], ['titl', 'the', 'mysteri', 'island', 'author', 'jule', 'vern', 'releas', 'date', 'april', 'ebook', 'most', 'recent', 'updat', 'june', 'languag', 'english', 'charact', 'set', 'encod', 'utf', 'produc', 'by', 'anthoni', 'matonak', 'sta'

Lematyzacja

In [ ]:

# Lematyzacja zdań
lemmatized_sentences = []
wnl = nltk.WordNetLemmatizer()
for sentence in stemmed_sentences:
    lemmatized_sentence = []
    for word in sentence:
        pos_tag = nltk.pos_tag([word])[0][1][0].lower()
        pos_tag = pos_tag if pos_tag in ['a', 's', 'r', 'n', 'v'] else None
        if pos_tag:
            lemmatized_word = wnl.lemmatize(word, pos=pos_tag)
            lemmatized_sentence.append(lemmatized_word)
    lemmatized_sentences.append(lemmatized_sentence)


print("Lematyzacja")
print(lemmatized_sentences)

Stopwords

In [ ]:

custom_stopwords = ['almost','anyon','anywher','be','pa','thi','mai','re','term','www','org','ar','before','have','licens','includ',
                    'project','restrict','whatsoev','awai','copi','give','before','check','law','not','again','author','sta','set',
                    'ebook','gutenberg','vern','jule','onlin','law','chapter','produc','utf','titl','updat','encod','date','even',
                    'everyth','everi','more','cannot','there','rd','hear','mysteri','island','cost','state','unit','befor','locat',
                    'countri','charact','languag','releas','rage','year','possibl','matonak','world','anthoni','contrari','wors',
                    'abov','vast','th','word','dash','nois','sake','drop','rise','ballast','fall','empti','doe','clock','resound',
                    'noh','fifth','eighteen','parallel','thii','cast','account','hundr','foieth','extend','land','furi','level',
                    'left','number','terribl','intermiss','forgotten','loud','weight','heav','descend','thei','publish','peopl','pas',
                    'sever','thousand','precipit','overhrown','wa','hi','use']  # Moje własne stopwords

clean_sentences = []
for sentence in lemmatized_sentences:
    clean_sentence = remove_stopwords(" ".join(sentence), stopwords=custom_stopwords)
    clean_sentence = clean_sentence.split()
    clean_sentences.append(clean_sentence)

# Wyświetlanie wyników stop words
print(clean_sentences)

clean_sentences_MI=clean_sentences


Korpus-macierz wystapien

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import TfidfModel

dictionary_MI = Dictionary(clean_sentences)

#Tworzenie korpusu
corpus_MI = [dictionary_MI.doc2bow(sentence) for sentence in clean_sentences]

#Model TF-IDF
tfidf = TfidfModel(corpus_MI)

print(dictionary_MI)

# tworzenie korpusu
corpus_MI = [dictionary_MI.doc2bow(sentence) for sentence in clean_sentences]
print(corpus_MI)

b)Podroz do wnetrza ziemi

In [ ]:
# Tekst książki
tekst = data_CE

# Wywołanie funkcji rozdzielającej na zdania
zdania = rozdziel_na_zdania(tekst)



Normalizacja

In [ ]:

czysty = [czysc_tekst(el) for el in zdania]

Tokenizacja

In [ ]:

tokenized_sentences = [simple_preprocess(sentence) for sentence in czysty]

# Wyświetlenie wyników tokenizacji
print(tokenized_sentences)

Stemming

In [ ]:

stemmer = PorterStemmer()
stemmed_sentences = [[stemmer.stem(word) for word in sentence] for sentence in tokenized_sentences]

print(stemmed_sentences)

Lematyzacja

In [ ]:

# Lematyzacja zdań
lemmatized_sentences = []
wnl = nltk.WordNetLemmatizer()
for sentence in stemmed_sentences:
    lemmatized_sentence = []
    for word in sentence:
        pos_tag = nltk.pos_tag([word])[0][1][0].lower()
        pos_tag = pos_tag if pos_tag in ['a', 's', 'r', 'n', 'v'] else None
        if pos_tag:
            lemmatized_word = wnl.lemmatize(word, pos=pos_tag)
            lemmatized_sentence.append(lemmatized_word)
    lemmatized_sentences.append(lemmatized_sentence)


print("Lematyzacja")
print(lemmatized_sentences)

Stopwords

In [ ]:

custom_stopwords = ['almost','anyon','anywher','be','pa','thi','mai','re','term','www','org','ar','before','have','licens','includ',
                    'project','restrict','whatsoev','awai','copi','give','before','check','law','not','again','author','sta','set',
                    'ebook','gutenberg','vern','jule','onlin','law','chapter','produc','utf','titl','updat','encod','date','even',
                    'everyth','everi','more','cannot','there','centr','eah','journei','ascii','charact','number','redactor','releas','ad','edit',
                    'novel','publish','trnaslat','onli','reprint','version','well','also','avail','cost','juli','languag','michaluk',
                    'norm','first','name','translat','write','rev','malleson','purpos','enter','chang','note','omit','refer','content',
                    'continu','convers','eider','lesson','make','meet','mine','alon','wai','uncl','tabl','reach','element','combat','disappoint',
                    'ap','abi','terrif','revers','recoveri','lose','mysteri','down','wolcott','work','griffith',
                    'farran','adventur','tave','complet','occur','look','ha','end','dai','back','abl','result','realiti','bitter','now',
                    'discoveri','commenc','interior','wonder','truli','think','thei','marri','so','mother','sister','wa','cri','ani','hi',"poion","use"]  # Moje własne stopwords

clean_sentences = []
for sentence in lemmatized_sentences:
    clean_sentence = remove_stopwords(" ".join(sentence), stopwords=custom_stopwords)
    clean_sentence = clean_sentence.split()
    clean_sentences.append(clean_sentence)

# Wyświetlanie wyników stop words
print(clean_sentences)
clean_sentences_WZ=clean_sentences

Tworzenie korpusu-macierz wystąpień

In [ ]:
dictionary_WZ = Dictionary(clean_sentences)

#Tworzenie korpusu
corpus_WZ = [dictionary_WZ.doc2bow(sentence) for sentence in clean_sentences]

#Model TF-IDF
tfidf = TfidfModel(corpus_WZ)

print(dictionary_WZ)

# tworzenie korpusu
corpus = [dictionary_WZ.doc2bow(sentence) for sentence in clean_sentences]
print(corpus_WZ)

c)Podroz dookola swiata


In [ ]:
# Tekst książki
tekst = data_AW

# Wywołanie funkcji rozdzielającej na zdania
zdania = rozdziel_na_zdania(tekst)

Normalizacja

In [ ]:

czysty = [czysc_tekst(el) for el in zdania]

Tokenizacja

In [ ]:

tokenized_sentences = [simple_preprocess(sentence) for sentence in czysty]

# Wyświetlenie wyników tokenizacji
print(tokenized_sentences)

Pierwsze stopwords

In [ ]:

clean_sentences = []
for sentence in tokenized_sentences:
    clean_sentence = remove_stopwords(" ".join(sentence))
    clean_sentence = clean_sentence.split()
    clean_sentences.append(clean_sentence)

# wyświetlenie wyników stop words    
print(clean_sentences)    

Stemming

In [ ]:

stemmer = PorterStemmer()
stemmed_sentences = [[stemmer.stem(word) for word in sentence] for sentence in clean_sentences]

print(stemmed_sentences)

Lematyzacja

In [ ]:

# Lematyzacja zdań
lemmatized_sentences = []
wnl = nltk.WordNetLemmatizer()
for sentence in stemmed_sentences:
    lemmatized_sentence = []
    for word in sentence:
        pos_tag = nltk.pos_tag([word])[0][1][0].lower()
        pos_tag = pos_tag if pos_tag in ['a', 's', 'r', 'n', 'v'] else None
        if pos_tag:
            lemmatized_word = wnl.lemmatize(word, pos=pos_tag)
            lemmatized_sentence.append(lemmatized_word)
    lemmatized_sentences.append(lemmatized_sentence)


print("Lematyzacja")
print(lemmatized_sentences)




Drugie stopwords

In [ ]:

custom_stopwords = ['almost','anyon','anywher','be','pa','thi','mai','re','term','www','org','ar','before','have','licens','includ',
                    'project','restrict','whatsoev','awai','copi','give','before','check','law','not','again','author','sta','set',
                    'ebook','gutenberg','vern','jule','onlin','law','chapter','produc','utf','titl','updat','encod','date','even',
                    'everyth','everi','more','cannot','there','centr','eah','journei','ascii','charact','number','redactor','releas','ad','edit',
                    'novel','publish','trnaslat','onli','reprint','version','well','also','avail','vii','vi','men','take','place','iv','iii','man','ii',
                    'languag','illustr','content','translat','locat','world','eighti','dai','cost','xi','talk','ix','viii','state','unit',
                    'countri','accept','convinc','ideal','convers','dear','appear','speci','detect','fix','impati','aid','think','see',
                    'length','xiv','receiv','proof','foun','favor','brave','xii','ventur','ensu','secur','mean','curiou','convey','glad',
                    'show','happen','xvii','understand','say','xvi','disgorg','bag','xv','beauti','companion','fabul','loss','prove','design',
                    'conveni','xxii','run','risk','reward','lose','xxi','face','xx','come','xix','xviii','voyag','xxv','mr','xxiv','long','xxiii',
                    'pocket','find','busi','thousand','useless','prudent','fogg','pai','xxvi','xxvii','slight','san','hour','mile',"xxviii","xiii","xxxi",
                    "xxxii","xxxv","xxx","xxxiii","xxxiv","xxix","xxxvi","use"

                    ]  # Moje własne stopwords

clean_sentences = []
for sentence in lemmatized_sentences:
    clean_sentence = remove_stopwords(" ".join(sentence), stopwords=custom_stopwords)
    clean_sentence = clean_sentence.split()
    clean_sentences.append(clean_sentence)

# Wyświetlanie wyników stop words
print(clean_sentences)
clean_sentences_DS=clean_sentences

Korpus-macierz wystąpień

In [ ]:
dictionary_DS = Dictionary(clean_sentences)

#Tworzenie korpusu
corpus_DS = [dictionary_DS.doc2bow(sentence) for sentence in clean_sentences]

#Model TF-IDF
tfidf = TfidfModel(corpus_DS)

print(dictionary_DS)

# tworzenie korpusu
corpus_DS = [dictionary_DS.doc2bow(sentence) for sentence in clean_sentences]
print(corpus_DS)

## II. Chmury słów

2.2 W tym punkcie zostaną przedstawione słowa ( przedstawiona za mocą chmr słów ) wystepuje najczęściej w danych książkach. Metoda ta została wybrana, aby spróbować się dowiedzieć coś więcej o fabule ksiązki bez czytania jej. Wybrane zostały najczęstsze słowa zamiast unikatowych, ponieważ w ten sposób dowiemy się więcej o fabule tym bardziej jeśli ich fabuła jest podobna. 

a) "Podróż na tajemniczą wyspę"

In [ ]:
# Podroz na tajemnicza- pierwszy dokument
import matplotlib.pyplot as plt

# Policz ilość wystąpień w korpusie
word_counts_MI = [sum(frequency for _, frequency in doc) for doc in corpus_MI]

# Sortowanie i ograniczenie do 50 najczęstszych wyników
sorted_word_counts_MI = sorted(word_counts_MI, reverse=True)[:70]


# Dodawanie podpisów dla kolumn
term_labels = [dictionary_MI[id] for id in dictionary_MI.keys()][:70]
termy_TW=term_labels



In [ ]:
# Konwertuj listę terminów na słownik z ich liczbą wystąpień
word_counts_dict_MI = {term: count for term, count in zip(termy_TW[:70], sorted_word_counts_MI[:70])}

# Inicjalizacja obiektu WordCloud
wordcloud_MI = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_counts_dict_MI)

# Wyświetlenie chmury słów
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud_MI, interpolation='bilinear')
plt.axis('off')
plt.title('Chmura słów - 70 najczęstszych terminów')
plt.show()


Analizując chmurę słów dla książki "Podróż na Tajemniczą Wyspę", widzimy wiele słów związanych z morzem/wodą jak np. "sea","storm", "wateri", "wave". Ale również słowa opisujące krajobraz jak: "coast","forest","mountain", "shore". Pojawiają się kierunki świata jak: "south", "noheast". Ale również słowa które wskazują na możliwy wątek jak połacznie słów "air " "crush", "trace", "catastroph". Pojawiają sie też środki transportu jak "car", "balloon".
Ale również miesiące w których prawdopodobnie odbywa się akcja książki "april", "june". Na podstawie tych słów możemy przypuszczać, że akcja książki działa się na morzu lub miała z nim jakiś wątek. Dodatkowo przez opisy krajobrazu i kierunki możemy stwierdzić, że bohaterzy podróżowali bądź opisywali miejsca króre widzieli i tak jak było wcześniej wspomniane ważnym wydarzeniem fabularnym mógł być wypadek samolotowy.  

b) "Podróż do wnętrza Ziemi"

In [ ]:

# Policz ilość wystąpień w korpusie
word_counts_WZ = [sum(frequency for _, frequency in doc) for doc in corpus_WZ]

# Sortowanie i ograniczenie do 50 najczęstszych wyników
sorted_word_counts_WZ = sorted(word_counts_WZ, reverse=True)[:70]

# Dodawanie podpisów dla kolumn
term_labels = [dictionary_WZ[id] for id in dictionary_WZ.keys()][:70]
termy_WZ=term_labels


In [ ]:

# Konwertuj listę terminów na słownik z ich liczbą wystąpień
word_counts_dict_WZ = {term: count for term, count in zip(termy_WZ[:70], sorted_word_counts_WZ[:70])}

# Inicjalizacja obiektu WordCloud
wordcloud_WZ = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_counts_dict_WZ)

# Wyświetlenie chmury słów
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud_WZ, interpolation='bilinear')
plt.axis('off')
plt.title('Chmura słów - 70 najczęstszych terminów')
plt.show()


W tej chmurze dla książki "Podróż do wnętrza Ziemi" na pierwszy plan wysywają się dwa słowa "england" i "decemb", możliwe że akcja dzieje się w Anglii, bądź są jakieś porównania do tego kraju, możwemy przypuszczać, że akcja dzieje się w grudniu. Słowa "raft" i "sea" mogą się połączyć i wskazywać na jakiś wątek fabuły. "Hunger" bohaterzy może nie mają dużo do jedzenia i są głodni. A słowa "tunnel", "climb","coal", że może akcja dzieje się pod ziemią, wspinają się do góry. Są również słowa takie jak: "chemistri","mineralog","laboratori", więc może jest coś związanego z odkrywcami/naukowcami. Najpopularniejszym słowem jest jednak "journey", które może wskazywać że bohaterzy są w podróży. Podsumowując słowa podpowiadają nam o podróży bohaterów wgłąb czegoś i ponownej wspinaczne, jest też wspomniane o jednym wydarzeniu fabularnym które ma rzeczywiście miejsce czym jest przeprawa przez morze na trwatwie. 

c) "W 80 dni dookoła Świata"

In [ ]:

# Policz ilość wystąpień w korpusie
word_counts_DS = [sum(frequency for _, frequency in doc) for doc in corpus_DS]

# Sortowanie i ograniczenie do 50 najczęstszych wyników
sorted_word_counts_DS = sorted(word_counts_DS, reverse=True)[:70]

# Dodawanie podpisów dla kolumn
term_labels = [dictionary_DS[id] for id in dictionary_DS.keys()][:70]
termy_OD=term_labels



In [ ]:
from wordcloud import WordCloud

# Konwertuj listę terminów na słownik z ich liczbą wystąpień
word_counts_dict_DS = {term: count for term, count in zip(termy_OD[:70], sorted_word_counts_DS[:70])}

# Inicjalizacja obiektu WordCloud
wordcloud_DS = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_counts_dict_DS)

# Wyświetlenie chmury słów
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud_DS, interpolation='bilinear')
plt.axis('off')
plt.title('Chmura słów - 70 najczęstszych terminów')
plt.show()


Słowa z trzeciej książki "W 80 dni dookoła świata", możemy wyróżnić klika miejsc takich jak "singapor", "hong kong", london". Miesiące w których odbywała się podróż "januari", "august". Można się dowiedzieć, że byli w podróży "travel", a nawet stwierdzić, że bohaterzy podróżowali pociągiem. Trochę też o krajobrazie "ocean","forest". Czas książki to prawdopodobnie wcześniejsze lata niż obecne stwierdzić to można po słowach "servant", "master". Co ciekawe na podstawie tych słów nie możemy wydobyć żadnego wyraźnego wątku fabularnego.

2.1 W tej części zostanie przedstawiona chmura słów ze słowami, które występują we wszystkich trzech książkach

In [ ]:

# Połączenie słów ze wszystkich trzech zbiorów
all_words = termy_TW + termy_WZ + termy_OD

# Policz ilość wystąpień każdego słowa
word_counts = {word: all_words.count(word) for word in set(all_words)}

# Sortowanie słów według liczby wystąpień
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:70]

# Tworzenie chmury słów
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(sorted_word_counts))

# Wyświetlanie chmury słów
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Chmura słów - 70 najpopularniejszych słów wspólnych dla 3 książek ')
plt.show()


Wszystkie 3 książki mają duże powiązanie z morzem "sea" i  oceanem "ocean", a więc również z wodą "water", tak samo z lasem "forest" jako krajobrazem. Występują tu podróże "travel", "voyag", "road", "railroad". Bohaterzy/Fabuła mają coś wspólnego z Anglią pojawiają się słowa "englishwoman", "london". Ae również miesiące akcji się pokrywają "march", "june","august","april". Jednak najpopularniejszym słowem jest "away" co można interpretować na kilka sposób np. że coś jest daleko lub że bohaterzy są daleko.

# III

W trzeciej części ksiązki zostaną podzielone na rozdziały i również przeprowadzone procesy normalizacji, a następnie zostanie przeprowadzone grupowanie a  samym końcu zostaną przedstawione wątki główne w rozdziałach. 

III*


In [ ]:
###################### Pierwsza ksiazka- "Podroz na Tajemnicza Wyspę"###################3
rozdziały = re.split(r'Chapter\s*\d+', data_MI)[1:]
# Usunięcie przypisów końcowych z ostatniego rozdziału
ostatni_rozdział = rozdziały[-1].split('END OF THE PROJECT')[0]
rozdziały[-1] = ostatni_rozdział

def czysc_tekst(tekst):
    temp = re.sub("\s{2,}", " ", tekst)
    temp = re.sub("(\r\n|\r|\n)", " ", temp)
    temp = temp.lower()
    temp = re.sub("rt", "", temp)
    temp = re.sub("&amp", "", temp)
    temp = re.sub("#[a-z,A-Z]*", "", temp)
    temp = re.sub("@\w+", "", temp)
    temp = re.sub("(f|ht)(tp)([^ ]*)", "", temp)
    temp = re.sub("http(s?)([^ ]*)", "", temp)
    temp = re.sub("[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", " ", temp)
    temp = re.sub("\d", "", temp)
    temp = re.sub("\s{2,}", " ", temp)
    temp = temp.strip()
    return temp

czyste_rozdziały = [czysc_tekst(rozdzielanie) for rozdzielanie in rozdziały]


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Pobranie stopwords w języku angielskim
stopwords = set(stopwords.words('english'))

# Inicjalizacja stemmera i lematyzatora
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

custom_stopwords=['chapter','wa','thi','hi','would','fogg','mr','eah','veri','one','repli','could',"cyru","pencroft","neb","hard","upon","like","day","herb","engin","spilett","minut","han","cri","hou",
                  "time","island","said","made","ayon","hous","ani","two","onli","sailor","well","must","uncl","great","go","man","see","professor","granit","repoer","without","appear","convict","captain","companion","gideon","vessel",'pa','might','colonist','return','brig',
                  'remain','even','might','land','rock','ask','befor','first','shall','water','foot','mile','make','think','ha','place','top','noth','still','howev','hour','tree','corral']
# Lematyzacja i stemming słów w rozdziałach oraz usunięcie stopwords
rozdziały_cleaned_MI = []
for rozdział in czyste_rozdziały:
    # Tokenizacja słów w rozdziale
    tokenized_words = simple_preprocess(rozdział)
    
    # Lematyzacja słów
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokenized_words]
    
    # Stemming słów
    stemmed_words = [stemmer.stem(word) for word in lemmatized_words]
    
    # Usunięcie stopwords
    cleaned_words = [word for word in stemmed_words if word not in stopwords and word not in custom_stopwords and len(word) > 1]
    cleaned_words = list(filter(None, cleaned_words))
    rozdziały_cleaned_MI.append(cleaned_words)
    # Usunięcie pustych pól z listy rozdziałów
rozdziały_cleaned_MI = [rozdział for rozdział in rozdziały_cleaned_MI if rozdział]


In [ ]:
######## Druga ksiazka- "Podroz do wnętrza Ziemi" ###############
content = data_CE

# Usuń tekst przed pierwszym wystąpieniem 'CHAPTER 44 THE JOURNEY ENDED'
chapter_start = "CHAPTER 44 THE JOURNEY ENDED"
content = content.split(chapter_start, 1)[-1].strip()

# Podziel tekst na rozdziały
chapters = re.split(r'(?:\n|^)(CHAPTER)\s+\d+', content)
chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

# Usuń wszystko po słowach "END OF THE PROJECT" w ramach każdego rozdziału
updated_chapters = []
end_of_project = "End of the Project"

for chapter in chapters:
    end_of_project_index = chapter.find(end_of_project)
    if end_of_project_index != -1:
        chapter = chapter[:end_of_project_index]
    updated_chapters.append(chapter.strip())


cleaned_chapters = [czysc_tekst(rozdzielanie) for rozdzielanie in updated_chapters]
custom_stopwords=['chapter','wa','thi','hi','would','fogg','mr','eah','veri','one','repli','could',"cyru","pencroft","neb","hard","upon","like","day","herb","engin","spilett","minut","han","cri","hou",
                  'passepaout','philea','fix','said','hour','time','aouda','onli','even','befor','go','master','seem','well','twenti','professor','without','howev','everi','man','way','uncl','see','great','two','mean','still','rock','ha','ani','appear',
                  'last','made','place','make','word','must','look','thought','began','moment','mighti','hundr','hand','whole','noth','continu','sea','foot','shall','boy','let','first','long','matter','guid','say','take','reach','water','much','never','back',
                  'follow','come','quit','side','came','head','iceland','']
# Lematyzacja i stemming słów w rozdziałach oraz usunięcie stopwords
rozdziały_cleaned_CE = []
for rozdział in cleaned_chapters:
    # Tokenizacja słów w rozdziale
    tokenized_words = simple_preprocess(rozdział)
    
    # Lematyzacja słów
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokenized_words]
    
    # Stemming słów
    stemmed_words = [stemmer.stem(word) for word in lemmatized_words]
    
    # Usunięcie stopwords
    cleaned_words = [word for word in stemmed_words if word not in stopwords and word not in custom_stopwords and len(word) > 1]
    
    rozdziały_cleaned_CE.append(cleaned_words)# Usunięcie pustych pól z listy rozdziałów
rozdziały_cleaned_CE = [rozdział for rozdział in rozdziały_cleaned_CE if rozdział]




In [ ]:
########Trzecia Ksiazka- "W 80 dni dookoła Świata" #################

content = data_AW

# Podziel tekst na rozdziały, korzystając z wyrażenia regularnego
chapters = re.split(r'\bCHAPTER [IVX]+\.', content)
chapters = [chapter.strip() for chapter in chapters if chapter.strip()]

# Usuń wszystko po słowach "END OF THE PROJECT GUTENBERG" w ostatnim rozdziale
end_of_project = "END OF THE PROJECT GUTENBERG"

if chapters:
    last_chapter = chapters[-1]
    end_of_project_index = last_chapter.find(end_of_project)
    if end_of_project_index != -1:
        last_chapter = last_chapter[:end_of_project_index]
    chapters[-1] = last_chapter.strip()


cleaned_chapters = [czysc_tekst(rozdzielanie) for rozdzielanie in chapters]
custom_stopwords=['chapter','wa','thi','hi','would','fogg','mr','eah','veri','one','repli','could',"cyru","pencroft","neb","hard","upon","like","day","herb","engin","spilett","minut","han","cri","hou",
                  'passepaout','philea','fix','said','hour','time','aouda','onli','even','befor','granit','pa','without','captain','even','might','twenti','mile','wind','train','sir','two','go','seem','well','steamer','franci','master','good',
                  'thousand','pass','hundr','made','reach','return','man','pass','thought','gentleman']
# Lematyzacja i stemming słów w rozdziałach oraz usunięcie stopwords
rozdziały_cleaned_AW = []
for rozdział in cleaned_chapters:
    # Tokenizacja słów w rozdziale
    tokenized_words = simple_preprocess(rozdział)
    
    # Lematyzacja słów
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokenized_words]
    
    # Stemming słów
    stemmed_words = [stemmer.stem(word) for word in lemmatized_words]
    
    # Usunięcie stopwords
    cleaned_words = [word for word in stemmed_words if word not in stopwords and word not in custom_stopwords and len(word) > 1]
    cleaned_words = list(filter(None, cleaned_words))

    rozdziały_cleaned_AW.append(cleaned_words)
# Usunięcie pustych pól z listy rozdziałów
rozdziały_cleaned_AW = [rozdział for rozdział in rozdziały_cleaned_AW if rozdział]

rozdziały_cleaned_AW = rozdziały_cleaned_AW[38:]

## tu się kończy normalizacja rozdzialow

Grupowanie 

In [ ]:

# Przygotowanie danych
all_chapters = rozdziały_cleaned_CE + rozdziały_cleaned_AW + rozdziały_cleaned_MI

# Połączenie rozdziałów w pojedyncze łańcuchy tekstowe
all_chapters_text = [' '.join(chapter) for chapter in all_chapters]

# Normalizacja
normalized_chapters = [chapter.lower() for chapter in all_chapters_text]

# Przygotowanie wektorów cech
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(normalized_chapters)

# Klasteryzacja hierarchiczna
clustering = AgglomerativeClustering(n_clusters=3)
cluster_labels = clustering.fit_predict(feature_vectors.toarray())

# Tworzenie drzewa dendrogramu
linked = linkage(feature_vectors.toarray(), 'ward')
plt.figure(figsize=(10, 6))
dendrogram(linked, orientation='top', distance_sort='descending', no_labels=True)
plt.title('Drzewo dendrogramu')
plt.xlabel('Indeksy rozdziałów')
plt.ylabel('Odległość')
plt.show()

# Analiza wyników klasteryzacji
for i, label in enumerate(cluster_labels):
    chapter_index = i  # Numer rozdziału
    book_name = ""  # Nazwa książki, do której należy rozdział

    if i < len(rozdziały_cleaned_CE):
        book_name = "Książka CE"
    elif i < len(rozdziały_cleaned_CE) + len(rozdziały_cleaned_AW):
        book_name = "Książka AW"
    else:
        book_name = "Książka MI"

    print(f"Rozdział o indeksie {chapter_index} z {book_name} należy do klastra {label+1}")


Wyniki z grupowania dla różnych liczb klastrów:
 

*   3- w tym przypadku wszystkie rozdziały w książkach przyporządkowane są do właściwych klastrów (książek) oprócz książki "Podróż na tajemniczą wyspę" gdzie kilka rozdziałów pod koniec zostało zgrupowane z rozdziałami "Podróż do wnętrza Ziemi"
*   4- wszystkie rozdziały z książki "Podróż do wnętrza Ziemi" stworzyły jedną grupę, tak samo rozdzaiły z książki "W 80 dni dookoła świata". Natomiast rozdziały z " Podróż na tajemniczą wyspę" podzieliły się na dwie grupy i nie można na pierwszy rzut oka znaleźć rzadnego wzorca, dlaczego tak się stało.

 


3.2. Modele wątków (topic
modelling) 

In [ ]:


dictionary_CE = corpora.Dictionary(rozdziały_cleaned_CE)
dictionary_AW = corpora.Dictionary(rozdziały_cleaned_AW)
dictionary_MI = corpora.Dictionary(rozdziały_cleaned_MI)

# Przygotowanie reprezentacji wektorowej dla każdego korpusu
bow_corpus_CE = [dictionary_CE.doc2bow(chapter) for chapter in rozdziały_cleaned_CE]
bow_corpus_AW = [dictionary_AW.doc2bow(chapter) for chapter in rozdziały_cleaned_AW]
bow_corpus_MI = [dictionary_MI.doc2bow(chapter) for chapter in rozdziały_cleaned_MI]

# Przeprowadzenie analizy LDA dla każdego korpusu
num_topics = 4  # Liczba tematów do zidentyfikowania

lda_CE = LdaModel(bow_corpus_CE, num_topics=num_topics, id2word=dictionary_CE, passes=10)
lda_AW = LdaModel(bow_corpus_AW, num_topics=num_topics, id2word=dictionary_AW, passes=10)
lda_MI = LdaModel(bow_corpus_MI, num_topics=num_topics, id2word=dictionary_MI, passes=10)

# Analiza wyników
def analyze_lda_results(lda_model, bow_corpus, dictionary):
    for i, chapter in enumerate(bow_corpus):
        print(f"Rozdział {i + 1}:")
        topics = lda_model.get_document_topics(chapter)
        for topic in topics:
            print(f"Temat {topic[0]}: {lda_model.print_topic(topic[0])}")



In [ ]:
# Przykład użycia
print("Analiza LDA dla książki 'CE':")
analyze_lda_results(lda_CE, bow_corpus_CE, dictionary_CE)



In [ ]:
print("Analiza LDA dla książki 'AW':")
analyze_lda_results(lda_AW, bow_corpus_AW, dictionary_AW)



In [ ]:
print("Analiza LDA dla książki 'MI':")
analyze_lda_results(lda_MI, bow_corpus_MI, dictionary_MI)


Podczas modelowania wątków pojawił się dość duży problem jakim były bardzo silne/dominujące wątki we wszystkich rozdziałach książek. Podjęta została próba mająca na celu sprawdzenia wątków, ale w całych książkach, jednak wyniki były takie same. Podjęto decyzję o pozostaniu w modelowaniu wątków rozdziałów. W celu polepszenia trochę analizy wątków pousuwano jako stopwords słowa dominujące. W momencie, gdy pojawiło się więcej różnych wątków przestano usuwać słowa.


Wnioski z analizy wątków:
Analizując rozdziały możemy je sobie sami pogrupować na te które miały taką samą bądź podobną tematykę/wątki, często pojawiają się po sobie 3/2 rozdziały z takimi samymi wątkami, może to mieć związek z kontynuacją wątku z poprzedniego rozdziału. 
Bardzo ciężko jest wyróżnić jakieś poszczególne wątki które miały miejsce w różnych rozdziałach książek i spróbować opisać co tam się mogło dziać. Nawet usuwanie słów dominujących nie pomogło w takim stopniu jak przypuszczano, dlatego aby nie tracić więcej informacji postanowiono o pozostawieniu dokumentów w takiej formie.







# IV.

W czwartej części zbudowany został klasyfikator z trzema różnymi metodami ważenia termów w celu porównaniu który najlepiej sobie poradził w przypadku analizowanych danych. Za klasy etykiet przyjęto tytuły książek. 

a)

In [ ]:


# Połączenie wszystkich rozdziałów w jedną listę i przekształcenie ich na tekst
all_text = [' '.join(chapter) for chapter in rozdziały_cleaned_CE + rozdziały_cleaned_MI + rozdziały_cleaned_AW]

labels_combined = ["Podroz do wnetrza ziemi"] * 44 + ["Podroz na nieznana wyspe"] * 62 + ["W 80 dni dookola swiata"] * 37

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(all_text, labels_combined, test_size=0.3, random_state=42)


# Inicjalizacja wektorów cech dla różnych metod ważenia termów
binary_vectorizer = CountVectorizer(binary=True)
log_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()

# Przekształcenie danych treningowych przy użyciu różnych metod ważenia termów
X_train_binary = binary_vectorizer.fit_transform(X_train)
X_train_log = log_vectorizer.fit_transform(X_train)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Budowa klasyfikatora dla metody ważenia binarnego
classifier_binary = MultinomialNB()
classifier_binary.fit(X_train_binary, y_train)

# Budowa klasyfikatora dla metody ważenia logarytmicznego
classifier_log = MultinomialNB()
classifier_log.fit(X_train_log, y_train)

# Budowa klasyfikatora dla metody TfIdf
classifier_tfidf = MultinomialNB()
classifier_tfidf.fit(X_train_tfidf, y_train)

# Przekształcenie danych testowych przy użyciu odpowiednich wektorów cech
X_test_binary = binary_vectorizer.transform(X_test)
X_test_log = log_vectorizer.transform(X_test)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Predykcja dla danych testowych i ocena klasyfikatorów
y_pred_binary = classifier_binary.predict(X_test_binary)
y_pred_log = classifier_log.predict(X_test_log)
y_pred_tfidf = classifier_tfidf.predict(X_test_tfidf)

# Wydrukowanie raportu klasyfikacji i macierzy pomyłek dla każdego klasyfikatora
print("Binarny:")
print(classification_report(y_test, y_pred_binary, zero_division=1))
print(confusion_matrix(y_test, y_pred_binary))

print("Logarytmiczny:")
print(classification_report(y_test, y_pred_log, zero_division=1))
print(confusion_matrix(y_test, y_pred_log))

print("TfIdf:")
print(classification_report(y_test, y_pred_tfidf, zero_division=1))
print(confusion_matrix(y_test, y_pred_tfidf))


Z klasyfikacji możemy dowiedzieć się że wszystkie ksiązki zostały pozytywnie zakwalifikowane. Trzy metody mają bardzo dobre wyniki, jedynie Tfdf ma trochę niższe od pozostałych dwóch, jednak wciąż są to bardzo dobre wyniki.

# V

W ostatniej części zastosowano wskaźnik polaryzacji nastroju rozdziałów dla każdej z książek, a następnie zanalizowano wyniki, które zostały przedstawione na wykresach.

In [ ]:

# Inicjalizacja analizatora sentymentu VADER
sia = SentimentIntensityAnalyzer()

# Funkcja do obliczania wskaźnika polaryzacji nastroju dla tekstu
def calculate_sentiment_polarity(text):
    sentiment_scores = sia.polarity_scores(text)
    polarity = sentiment_scores["compound"]
    return polarity

# Obliczenie wskaźnika polaryzacji nastroju dla każdego rozdziału w książkach
chapter_polarities = []

# Książka 1
for idx, chapter in enumerate(rozdziały_cleaned_CE, start=1):
    chapter_text = ' '.join(chapter)
    chapter_polarity = calculate_sentiment_polarity(chapter_text)
    chapter_polarities.append(('Podroz do wnetrza ziemi', idx, chapter_polarity))

# Książka 2
for idx, chapter in enumerate(rozdziały_cleaned_MI, start=1):
    chapter_text = ' '.join(chapter)
    chapter_polarity = calculate_sentiment_polarity(chapter_text)
    chapter_polarities.append(('Podroz na Tajemnicza wyspe', idx, chapter_polarity))

# Książka 3
for idx, chapter in enumerate(rozdziały_cleaned_AW, start=1):
    chapter_text = ' '.join(chapter)
    chapter_polarity = calculate_sentiment_polarity(chapter_text)
    chapter_polarities.append(('W 80 dni dookola swiata', idx, chapter_polarity))


In [ ]:
import matplotlib.pyplot as plt

# Podział danych dla każdej książki
book1_polarities = [(idx, polarity) for (book, idx, polarity) in chapter_polarities if book == 'Podroz do wnetrza ziemi']
book2_polarities = [(idx, polarity) for (book, idx, polarity) in chapter_polarities if book == 'Podroz na Tajemnicza wyspe']
book3_polarities = [(idx, polarity) for (book, idx, polarity) in chapter_polarities if book == 'W 80 dni dookola swiata']

# Konfiguracja wykresów
fig, axs = plt.subplots(3, 1, figsize=(8, 12))

# Wykres dla książki 1
axs[0].plot(*zip(*book1_polarities))
axs[0].set_title('Podroz do wnetrza ziemi')

# Wykres dla książki 2
axs[1].plot(*zip(*book2_polarities))
axs[1].set_title('Podroz na Tajemnicza wyspe')

# Wykres dla książki 3
axs[2].plot(*zip(*book3_polarities))
axs[2].set_title('W 80 dni dookola swiata')

# Ustawienie oznaczeń osi dla ostatniego wykresu
axs[2].set_xlabel('Indeks rozdziału')
axs[2].set_ylabel('Wskaźnik polaryzacji nastroju')

# Dostosowanie odstępów między wykresami
plt.tight_layout()

# Wyświetlenie wykresów
plt.show()


Z polaryzacji nastoroju książek (ich rozdziałow) możemy wywnioskować, że:
- "Podróż na tajemniczą wyspę"- początek jest negatywny i dopiero buduje się pozytywny nastrój, który bardzo szybko ponownie opada do nautralnego, następnie znów w wzrasta i opada, środek książki jest w miarę spokojny, ale w 20 ostatnich rozdzaiłach ponownie pojawiają się częste zmiany nastroju. Ogólnie możemy zauważyć że początek i koniec mają podobne szybkie zmiany nastroju.
- "W 80 dni dookoła świata" i "Podróż do wnętrza Ziemi" mają bardzo podobne zmiany nastrojów, choć ten pierwszy tytuł ma mniej tych bardzo negatywnych nastrojów od drugiego 

## **Wnioski końcowe:**

Z całości przeprowadzonej analizy można wysunąć następujące wnioski:
Najlepiej do opisania co się może dziać w książkach i ogólny opis fabuły mogliśmy zoabaczyć poprzez chmury słów. Tam najłatwiej i najlepiej było to widoczne. Książki mają bardzo dominujące wątki w swoich rozdziałach i możemy przypuszczać, że w związku z tym tak dobrze rozdzaiły zostały zaklasyfikowane do poprawnych książek. Podobnie podczas grupowania. Natomiast z analizy nastroju możemy wywnioskować, że są to raczej pozytywne książki które raz na jakiś czas (może aby zbudować napięcie) mają mniej pozytywne nastroje rozdziałów.